In [2]:
import torch, cv2, os# 原340行
import numpy as np
np.random.seed(0)
from tqdm import trange
from PIL import Image
from nets.yolo import YoloBody
from pytorch_grad_cam import GradCAMPlusPlus,GradCAM,XGradCAM,EigenCAM,RandomCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

def letterbox(im,new_shape=(640,640),color=(114,114,114),不填充=True):
    shape = im.shape[:2]  # current shape [height, width]
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]
    if 不填充:dw,dh=np.mod(dw,32),np.mod(dh,32) #仅将最长边拉到640
    im = cv2.resize(im, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(dh/2 - 0.1)), int(round(dh/2 + 0.1))
    left, right = int(round(dw/2 - 0.1)), int(round(dw/2 + 0.1))
    return cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color), im# add border

class ActivationsAndGradients:#从目标中间层提取激活值并注册梯度
    def __init__(self, model, target_layers, reshape_transform):
        self.model = model
        self.gradients,self.activations,self.handles = [],[],[]
        for target_layer in target_layers:#为每一层注册前向hook来保存激活值和梯度
            self.handles.append(target_layer.register_forward_hook(self.save_activation)); self.handles.append(target_layer.register_forward_hook(self.save_gradient))
    def save_activation(self, module, input, output):#模型出入这些好像是自动捕捉的
        self.activations.append(output.cpu().detach())
    def save_gradient(self, module, input, output):
        def _store_grad(grad):#only register hooks on tensor requires grad.
            self.gradients = [grad.cpu().detach()] + self.gradients
        output.register_hook(_store_grad)
    def 后处理(self, boxes_,logits_):#此后处理非nms！
        sorted, indices = torch.sort(logits_.max(1)[0], descending=True)
        return torch.transpose(logits_[0], dim0=0, dim1=1)[indices[0]], torch.transpose(boxes_[0], dim0=0, dim1=1)[indices[0]]
    def __call__(self, x):
        self.gradients,self.activations = [],[]
        return [self.后处理(self.model(x)[0],self.model(x)[1])]
    def release(self):
        for handle in self.handles:handle.remove()

class yolov8_target(torch.nn.Module):
    def __init__(self, ouput_type, conf, ratio) -> None:
        super().__init__()
        self.ouput_type = ouput_type
        self.conf = conf
        self.ratio = ratio
    def forward(self, data):
        post_result, pre_post_boxes, result = data[0],data[1],[]
        for i in trange(int(post_result.size(0) * self.ratio)):
            if float(post_result[i].max()) < self.conf: break
            if self.ouput_type == 'class' or self.ouput_type == 'all':
                result.append(post_result[i].max())
            elif self.ouput_type == 'box' or self.ouput_type == 'all':
                for j in range(4):result.append(pre_post_boxes[i, j])
        return sum(result)#浓缩为一个整体

class yolov8_heatmap:
    def process(self, img_path, save_path,不填充):
        if not img_path.endswith('.jpg'):return#可能是"images/.ipynb_checkpoints"
        img = letterbox(cv2.imread(img_path),不填充=不填充)[0]
        img = np.float32(cv2.cvtColor(img,cv2.COLOR_BGR2RGB)) / 255.0
        try:grayscale_cam = self.method(torch.from_numpy(np.transpose(img,axes=[2,0,1])).unsqueeze(0).to(self.device), [self.target])
        except AttributeError as e:return
        cam_image = show_cam_on_image(img, grayscale_cam[0, :], use_rgb=True)
        Image.fromarray(cam_image).save(save_path)
    def __call__(self, img_path, save_path,不填充):
        if os.path.isdir(img_path):#是目录则在保存路径下存为各名，否则一张结果图
            os.makedirs(save_path, exist_ok=True)#建立新的保存地址存放多张结果图
            for img_path_ in os.listdir(img_path):
                self.process(f'{img_path}/{img_path_}',f'{save_path}/{img_path_}',不填充)
        else:self.process(img_path, f'result.png',不填充)
    def __init__(self,weight,device,method,layer,backward_type,信阈,ratio):
        model = YoloBody([640, 640], 7, 'n', pretrained=False)
        pretrained_dict = torch.load(weight, map_location='cpu')
        model.load_state_dict(pretrained_dict);model.eval()
        for p in model.parameters(): p.requires_grad_(True)#梯降才能返梯度
        target = yolov8_target(backward_type, 信阈, ratio)
        self.target_layers = eval(layer)
        target_layers = [model.backbone.dark3.头]#此处为网络具体层的输出，用"."嵌套，若为序列则可用[0~序列长度]如[1]代替".头"
        #######  网络特定层设置,默认为bak的".头",nek则"[0]",原网为"[1]"  #######
        # print(model) #若不知取何层可打印整个网络架构
        method = eval(method)(model, target_layers)
        method.activations_and_grads = ActivationsAndGradients(model, target_layers, None)
        self.__dict__.update(locals())
def get_params(): #bak模式不用改，其他模式详见下（注意还需改上面的特定层）
    params={'weight': '出简701.pth',#默认是骨网改进的权重，若颈网则用'无注733.pth'还需在最后一句中注明不填充为True
        # 'weight': 'model_data/b基础633.pth', #用原来的网络解除此注释
        'device':'cpu','method':'GradCAM',#GradCAMPlusPlus,XGradCAM,EigenCAM
        'layer':'[0,1]', #这是yolo源码的任何层均有对应序号，但bub码只能在target_layers中设定，见向上箭头标记处
        'backward_type': 'box', # class, box, all
        '信阈': 0.00001,'ratio': 0.1} # 0.2//0.02-0.1 ，取样占比
    return params

if __name__ == '__main__':
    model = yolov8_heatmap(**get_params())  # False 颈网改为假↓
    model('VOCdevkit/VOC2007/JPEGImages/4.jpg','result',不填充=True) #为方便仅处理单图，若需多图处理将前号内改为多图所在文件夹，生成热力图均见result文件夹，还可改为效果较好的14.jpg/15.jpg/127.jpg(bek适用)。

initialize network with normal type


 24%|██▍       | 133/546 [00:00<00:00, 20736.10it/s]
